In [1]:
%run utils/devtools.py

In [75]:
%reloadmypkg utils

from openpyxl import load_workbook
import os
import json
from tqdm import tqdm
from utils.standardise_url import expand_url, strip_query_params
from utils.enums import ProblemPlatform
from utils.url2platform import tutorial_url2platform, problem_url2platform
from utils.parse4db import problems_parse4db, tutorials_parse4db

✅ Reloaded package 'utils' and its submodules.


In [3]:
file_path = "../raw-data/love-babbar/450.xlsx"
export_file = "../cleaned-data/lb/450.json"

In [82]:
problems_urls_titles = []
tutorials_urls_titles = []

wb = load_workbook(file_path)
ws = wb.active
for row in ws.iter_rows(min_row=6):
    cell = row[1]
    if cell.hyperlink:
        problems_urls_titles.append({"url": cell.hyperlink.target, "title": cell.value})
    elif cell.value is not None and cell.value != '<->' and isinstance(cell.value, str):
        tutorials_urls_titles.append({"title": cell.value, "url": None})

In [83]:
print(len(problems_urls_titles))
print(problems_urls_titles[:3])

445
[{'url': 'https://www.geeksforgeeks.org/write-a-program-to-reverse-an-array-or-string/', 'title': 'Reverse the array'}, {'url': 'https://www.geeksforgeeks.org/maximum-and-minimum-in-an-array/', 'title': 'Find the maximum and minimum element in an array'}, {'url': 'https://practice.geeksforgeeks.org/problems/kth-smallest-element/0', 'title': 'Find the "Kth" max and min element of an array '}]


In [84]:
print(len(tutorials_urls_titles))

3


In [85]:
print(f"{0}: {tutorials_urls_titles[0]}")
print(f"{1}: {tutorials_urls_titles[1]}")
print(f"{2}: {tutorials_urls_titles[2]}")

0: {'title': 'Why strings are immutable in Java?', 'url': None}
1: {'title': 'Can we reverse a linked list in less than O(n) ?', 'url': None}
2: {'title': 'Why Quicksort is preferred for. Arrays and Merge Sort for LinkedLists ?', 'url': None}


In [86]:
tutorials_urls_titles[0]["url"] = "https://www.scaler.com/topics/why-string-is-immutable-in-java/"
tutorials_urls_titles[1]["url"] = "https://www.naukri.com/code360/library/is-it-possible-to-reverse-a-linked-list-in-less-than-o-n"
tutorials_urls_titles[2]["url"] = "https://www.naukri.com/code360/library/why-is-quick-sort-preferred-for-arrays-and-merge-sort-for-linked-lists"
tutorials_urls_titles[2]["title"] = "Why is quick sort preferred for arrays and merge sort for linked lists?"

In [87]:
tutorials_urls_titles

[{'title': 'Why strings are immutable in Java?',
  'url': 'https://www.scaler.com/topics/why-string-is-immutable-in-java/'},
 {'title': 'Can we reverse a linked list in less than O(n) ?',
  'url': 'https://www.naukri.com/code360/library/is-it-possible-to-reverse-a-linked-list-in-less-than-o-n'},
 {'title': 'Why is quick sort preferred for arrays and merge sort for linked lists?',
  'url': 'https://www.naukri.com/code360/library/why-is-quick-sort-preferred-for-arrays-and-merge-sort-for-linked-lists'}]

In [88]:
# Check the links which are duplicated

from collections import defaultdict

url_to_indexes = defaultdict(list)

for idx, item in enumerate(problems_urls_titles):
    url = item.get("url")
    if url:
        url_to_indexes[url].append(idx)

for url, indexes in url_to_indexes.items():
    if len(indexes) > 1:
        print(f"Duplicate URL: {url}")
        print(f"Indexes: {indexes}")
        for index in indexes:
            print(f"{index}: {problems_urls_titles[index]}")
        print("--------------------------------------------------------------------------------------------------------------------------------------------")

Duplicate URL: https://practice.geeksforgeeks.org/problems/kadanes-algorithm/0
Indexes: [7, 12, 409]
7: {'url': 'https://practice.geeksforgeeks.org/problems/kadanes-algorithm/0', 'title': 'find Largest sum contiguous Subarray [V. IMP]'}
12: {'url': 'https://practice.geeksforgeeks.org/problems/kadanes-algorithm/0', 'title': "Kadane's Algo [V.V.V.V.V IMP]"}
409: {'url': 'https://practice.geeksforgeeks.org/problems/kadanes-algorithm/0', 'title': 'LargestSum Contiguous Subarray [V>V>V>V IMP ]'}
--------------------------------------------------------------------------------------------------------------------------------------------
Duplicate URL: https://practice.geeksforgeeks.org/problems/minimum-number-of-jumps/0
Indexes: [9, 403]
9: {'url': 'https://practice.geeksforgeeks.org/problems/minimum-number-of-jumps/0', 'title': 'Minimum no. of Jumps to reach end of an array'}
403: {'url': 'https://practice.geeksforgeeks.org/problems/minimum-number-of-jumps/0', 'title': 'Minimum number of jump

In [89]:
# Links which had issues later in the program

indexes = [11, 78, 284, 106, 109, 123, 281, 269, 163, 164, 165, 325]
duped = []
for index in indexes:
    duped.append(problems_urls_titles[index])
duped

[{'url': 'https://practice.geeksforgeeks.org/problems/merge-two-sorted-arrays5135/1',
  'title': 'Merge 2 sorted arrays without using Extra space.'},
 {'url': 'https://practice.geeksforgeeks.org/problems/rearrange-characters/0',
  'title': 'Rearrange characters in a string such that no two adjacent are same'},
 {'url': 'https://practice.geeksforgeeks.org/problems/overlapping-intervals/0',
  'title': 'Merge Overlapping Intervals'},
 {'url': 'https://www.hackerearth.com/practice/algorithms/searching/binary-search/practice-problems/algorithm/bishu-and-soldiers/',
  'title': 'Bishu and Soldiers'},
 {'url': 'http://theoryofprogramming.com/2017/12/16/find-pivot-element-sorted-rotated-array/',
  'title': 'Find pivot element in a sorted array'},
 {'url': 'https://www.baeldung.com/java-sorting-arrays-with-repeated-entries',
  'title': 'Partitioning and Sorting Arrays with Many Repeated Entries'},
 {'url': 'https://stackoverflow.com/questions/45130465/inserting-at-the-end-of-stack',
  'title': '

In [90]:
len(duped)

12

In [93]:
# Dedupe

seen = set()
unique_problems = []
for item in problems_urls_titles:
    url = item.get("url")
    if url not in seen:
        seen.add(url)
        unique_problems.append(item)
print(f"Deleted {len(problems_urls_titles) - len(unique_problems)} duplicate problems.")
print(f"Previous number of total elements in the sheet was {len(problems_urls_titles) + len(tutorials_urls_titles)}.")
print(f"Current number of total elements in the sheet is {len(unique_problems) + len(tutorials_urls_titles)}.")

Deleted 29 duplicate problems.
Previous number of total elements in the sheet was 448.
Current number of total elements in the sheet is 419.


In [94]:
problems_urls_titles = unique_problems

In [95]:
len(problems_urls_titles)

416

In [96]:
for index, item in enumerate(problems_urls_titles):
    url = item['url']
    for d in duped:
        if d['url'] == url:
            print(f"{index}: {item}")

11: {'url': 'https://practice.geeksforgeeks.org/problems/merge-two-sorted-arrays5135/1', 'title': 'Merge 2 sorted arrays without using Extra space.'}
77: {'url': 'https://practice.geeksforgeeks.org/problems/rearrange-characters/0', 'title': 'Rearrange characters in a string such that no two adjacent are same'}
104: {'url': 'https://www.hackerearth.com/practice/algorithms/searching/binary-search/practice-problems/algorithm/bishu-and-soldiers/', 'title': 'Bishu and Soldiers'}
107: {'url': 'http://theoryofprogramming.com/2017/12/16/find-pivot-element-sorted-rotated-array/', 'title': 'Find pivot element in a sorted array'}
119: {'url': 'https://www.baeldung.com/java-sorting-arrays-with-repeated-entries', 'title': 'Partitioning and Sorting Arrays with Many Repeated Entries'}
159: {'url': 'https://www.techiedelight.com/inorder-tree-traversal-iterative-recursive/', 'title': 'Inorder Traversal of a tree both using recursion and Iteration'}
160: {'url': 'https://www.techiedelight.com/preorder-t

In [97]:
# Fix broken links in the sheet
problems_urls_titles[11]["url"] = "https://www.geeksforgeeks.org/problems/merge-two-sorted-arrays-1587115620/0"
problems_urls_titles[77]["url"] = "https://www.geeksforgeeks.org/problems/rearrange-characters4649/1"
problems_urls_titles[276]["url"] = "https://www.geeksforgeeks.org/problems/overlapping-intervals--170633/0"
problems_urls_titles[104]["url"] = "https://www.hackerearth.com/problem/algorithm/bishu-and-soldiers-227/"

# Replacing links with better links
problems_urls_titles[107]["url"] = "https://leetcode.com/problems/search-in-rotated-sorted-array/"
problems_urls_titles[119]["url"] = "https://www.naukri.com/code360/problems/partitioning-and-sorting-arrays-with-many-repeated-entries_1170515"
problems_urls_titles[273]["url"] = "https://www.naukri.com/code360/problems/insert-an-element-at-its-bottom-in-a-given-stack_1171166"
problems_urls_titles[262]["url"] = "https://www.programiz.com/dsa/stack"
problems_urls_titles[159]["url"] = "https://leetcode.com/problems/binary-tree-inorder-traversal/"
problems_urls_titles[160]["url"] = "https://leetcode.com/problems/binary-tree-preorder-traversal/"
problems_urls_titles[161]["url"] = "https://leetcode.com/problems/binary-tree-postorder-traversal/"
problems_urls_titles[313]["url"] = "https://www.naukri.com/code360/problems/create-a-graph-and-print-it_1214551"

In [98]:
real_problems_urls_titles = []
for index in tqdm(range(len(problems_urls_titles)), desc="Expanding LB450 URLS", ncols=100):
    item = problems_urls_titles[index]
    url = item["url"]
    title = item["title"]
    if "bit.ly" in url or "codingninjas.com/codestudio" in url or "codingninjas.com/studio" in url:
        url = expand_url(url)
    platform = problem_url2platform(url)
    if platform != ProblemPlatform.UNKNOWN:
        url = strip_query_params(url)
        real_problems_urls_titles.append({"url": url, "title": title})
    else:
        tutorials_urls_titles.append({"url": url, "title": title})

Expanding LB450 URLS: 100%|████████████████████████████████████| 416/416 [00:00<00:00, 25941.58it/s]

Unknown platform for url: https://www.geeksforgeeks.org/write-a-program-to-reverse-an-array-or-string/
Unknown platform for url: https://www.geeksforgeeks.org/maximum-and-minimum-in-an-array/
Unknown platform for url: https://www.geeksforgeeks.org/move-negative-numbers-beginning-positive-end-constant-extra-space/
Unknown platform for url: https://www.geeksforgeeks.org/rearrange-array-alternating-positive-negative-items-o1-extra-space/
Unknown platform for url: https://www.geeksforgeeks.org/given-an-array-of-of-size-n-finds-all-the-elements-that-appear-more-than-nk-times/
Unknown platform for url: https://www.geeksforgeeks.org/maximum-profit-by-buying-and-selling-a-share-at-most-twice/
Unknown platform for url: https://www.geeksforgeeks.org/median-of-two-sorted-arrays-of-different-sizes/
Unknown platform for url: https://www.geeksforgeeks.org/find-a-specific-pair-in-matrix/
Unknown platform for url: https://www.geeksforgeeks.org/rotate-a-matrix-by-90-degree-in-clockwise-direction-withou

In [99]:
len(real_problems_urls_titles)

273

In [100]:
problems_urls_titles = real_problems_urls_titles

In [101]:
len(tutorials_urls_titles)

146

In [102]:
len(problems_urls_titles) + len(tutorials_urls_titles)

419

In [103]:
urls = [item["url"] for item in problems_urls_titles]

In [104]:
len(urls)

273

In [105]:
# Count the number of problems in each platform
from collections import Counter

platforms = [problem_url2platform(url).value for url in urls]
count = Counter(platforms)
count

Counter({'GFG': 229, 'LC': 25, 'SPOJ': 11, 'HE': 4, 'C360': 3, 'HR': 1})

In [106]:
parsed_data = problems_parse4db(urls, "lb-450-progress.json")
len(parsed_data)

Processing URLs:   0%|                                                      | 0/273 [00:00<?, ?it/s]

Processing URLs:  29%|████████████▊                                | 78/273 [01:59<04:56,  1.52s/it]

No slug found for url: https://www.hackerearth.com/problem/algorithm/bishu-and-soldiers-227/


Processing URLs:  29%|█████████████                                | 79/273 [02:00<04:14,  1.31s/it]

No slug found for url: https://www.hackerearth.com/practice/algorithms/searching/binary-search/practice-problems/algorithm/rasta-and-kheshtak/


...

No slug found for url: https://www.hackerrank.com/challenges/journey-to-the-moon/problem


Processing URLs:  85%|█████████████████████████████████████▍      | 232/273 [05:29<00:55,  1.35s/it]

No slug found for url: https://www.hackerearth.com/practice/algorithms/graphs/topological-sort/practice-problems/algorithm/oliver-and-the-game-3/


Processing URLs: 100%|████████████████████████████████████████████| 273/273 [06:24<00:00,  1.41s/it]


273

In [107]:
parsed_data[:3]

[{'id': '701747GFG',
  'type': 'problem',
  'title': 'Kth Smallest',
  'platform': 'GFG',
  'href': 'https://practice.geeksforgeeks.org/problems/kth-smallest-element/0'},
 {'id': '702382GFG',
  'type': 'problem',
  'title': 'Sort 0s, 1s and 2s',
  'platform': 'GFG',
  'href': 'https://practice.geeksforgeeks.org/problems/sort-an-array-of-0s-1s-and-2s/0'},
 {'id': '701730GFG',
  'type': 'problem',
  'title': 'Union of Arrays with Duplicates',
  'platform': 'GFG',
  'href': 'https://practice.geeksforgeeks.org/problems/union-of-two-arrays/0'}]

In [108]:
problems_parsed = [{**parsed_data[i], "title": problems_urls_titles[i]["title"]} for i in range(len(parsed_data))]

In [109]:
len(problems_parsed)

273

In [110]:
problems_parsed[:3]

[{'id': '701747GFG',
  'type': 'problem',
  'title': 'Find the "Kth" max and min element of an array ',
  'platform': 'GFG',
  'href': 'https://practice.geeksforgeeks.org/problems/kth-smallest-element/0'},
 {'id': '702382GFG',
  'type': 'problem',
  'title': 'Given an array which consists of only 0, 1 and 2. Sort the array without using any sorting algo',
  'platform': 'GFG',
  'href': 'https://practice.geeksforgeeks.org/problems/sort-an-array-of-0s-1s-and-2s/0'},
 {'id': '701730GFG',
  'type': 'problem',
  'title': 'Find the Union and Intersection of the two sorted arrays.',
  'platform': 'GFG',
  'href': 'https://practice.geeksforgeeks.org/problems/union-of-two-arrays/0'}]

In [111]:
tutorials_parsed = tutorials_parse4db(tutorials_urls_titles)

Processing URLs: 100%|██████████████████████████████████████████| 146/146 [00:00<00:00, 3954.54it/s]

No slug found for url: https://www.scaler.com/topics/why-string-is-immutable-in-java/
No slug found for url: https://www.naukri.com/code360/library/is-it-possible-to-reverse-a-linked-list-in-less-than-o-n
No slug found for url: https://www.naukri.com/code360/library/why-is-quick-sort-preferred-for-arrays-and-merge-sort-for-linked-lists
No slug found for url: https://www.programiz.com/java-programming/examples/check-valid-shuffle-of-strings
No slug found for url: https://www.programiz.com/dsa/stack


In [112]:
len(tutorials_parsed)

146

In [113]:
tutorials_parsed[:3]

[{'id': '4ebd0208-8328-5d69-8c44-ec50939c0967SCALER',
  'type': 'tutorial',
  'title': 'Why strings are immutable in Java?',
  'platform': 'SCALER',
  'href': 'https://www.scaler.com/topics/why-string-is-immutable-in-java/'},
 {'id': '4ebd0208-8328-5d69-8c44-ec50939c0967C360',
  'type': 'tutorial',
  'title': 'Can we reverse a linked list in less than O(n) ?',
  'platform': 'C360',
  'href': 'https://www.naukri.com/code360/library/is-it-possible-to-reverse-a-linked-list-in-less-than-o-n'},
 {'id': '4ebd0208-8328-5d69-8c44-ec50939c0967C360',
  'type': 'tutorial',
  'title': 'Why is quick sort preferred for arrays and merge sort for linked lists?',
  'platform': 'C360',
  'href': 'https://www.naukri.com/code360/library/why-is-quick-sort-preferred-for-arrays-and-merge-sort-for-linked-lists'}]

In [114]:
parsed_merged = problems_parsed + tutorials_parsed

In [115]:
len(parsed_merged)

419

In [116]:
parsed_merged[:3]

[{'id': '701747GFG',
  'type': 'problem',
  'title': 'Find the "Kth" max and min element of an array ',
  'platform': 'GFG',
  'href': 'https://practice.geeksforgeeks.org/problems/kth-smallest-element/0'},
 {'id': '702382GFG',
  'type': 'problem',
  'title': 'Given an array which consists of only 0, 1 and 2. Sort the array without using any sorting algo',
  'platform': 'GFG',
  'href': 'https://practice.geeksforgeeks.org/problems/sort-an-array-of-0s-1s-and-2s/0'},
 {'id': '701730GFG',
  'type': 'problem',
  'title': 'Find the Union and Intersection of the two sorted arrays.',
  'platform': 'GFG',
  'href': 'https://practice.geeksforgeeks.org/problems/union-of-two-arrays/0'}]

In [117]:
os.makedirs(os.path.dirname(export_file), exist_ok=True)

In [118]:
with open(export_file, "w", encoding="utf-8") as f:
    json.dump(parsed_merged, f, ensure_ascii=False, indent=4)